In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 2    # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_exact_50-2.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:37:18<00:00,  9.44s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      2.594323               0.0      9.437726
std       3.359221               0.0      9.982172
min       0.000433               0.0      0.371946
25%       0.171067               0.0      3.400637
50%       1.217010               0.0      6.230896
75%       3.839457               0.0     10.999128
max      20.375223               0.0     60.679579
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rnd_50-2.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:24:26<00:00,  8.67s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      7.725639          0.099861      8.661723
std       4.482826          0.589254     15.877026
min       0.058280          0.000000      0.653168
25%       4.481548          0.000000      0.887213
50%       7.139306          0.000000      1.007757
75%      10.094171          0.000000      7.223290
max      27.025234          9.499012     60.751676
Number of infeasible solution: 91


## Heuristic - RENS

In [10]:
from src.heuristic import rens

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = rens(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rens_50-2.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:51:33<00:00, 10.29s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      4.763016          0.052960     10.288159
std      39.801592          0.421262     15.980096
min       0.000433          0.000000      1.360606
25%       0.206899          0.000000      2.132333
50%       1.286711          0.000000      2.674155
75%       4.159335          0.000000      9.040735
max    1149.403532          5.094683     68.908361
Number of infeasible solution: 23


## Heuristic - N1

In [12]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-2.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:26<00:00,  6.80it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean    532.157325               0.0      0.146008
std     363.175683               0.0      0.045320
min      75.030866               0.0      0.105247
25%     186.276981               0.0      0.121865
50%     470.379688               0.0      0.124276
75%     889.177330               0.0      0.154756
max    1217.174467               0.0      0.355438
Number of infeasible solution: 0


## Heuristic - N3

In [14]:
model_heur = model.first_solution_heuristic(nodes_limit=3)

In [15]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n3_50-2.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:35<00:00,  2.20it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean    124.889115               0.0      0.454240
std     238.139014               0.0      2.342601
min       0.037609               0.0      0.108486
25%      10.780444               0.0      0.139297
50%      29.830091               0.0      0.169687
75%      78.745917               0.0      0.230675
max     972.548668               0.0     52.469645
Number of infeasible solution: 0


## Learnable Rounding

In [16]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [17]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# define Rosenbrock objective functions and constraints for both problem types
obj_rel, constrs_rel = nmRosenbrock(["x"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
obj_rnd, constrs_rnd = nmRosenbrock(["x_rnd"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = [smap, rnd]
loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
problem = nm.problem.Problem(components, loss)

In [18]:
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(problem.parameters(), lr=lr)
# create a trainer for the problem
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test, optimizer, 
                            epochs=epochs, patience=patience, warmup=warmup)
# training for the rounding problem
best_model = trainer.train()
# load best model dict
problem.load_state_dict(best_model)

epoch: 0  train_loss: 141.621337890625
epoch: 1  train_loss: 54.17548370361328
epoch: 2  train_loss: 36.103633880615234
epoch: 3  train_loss: 29.21131134033203
epoch: 4  train_loss: 25.601943969726562
epoch: 5  train_loss: 23.98379898071289
epoch: 6  train_loss: 21.947715759277344
epoch: 7  train_loss: 20.99851417541504
epoch: 8  train_loss: 19.32303237915039
epoch: 9  train_loss: 18.250574111938477
epoch: 10  train_loss: 16.494781494140625
epoch: 11  train_loss: 16.3751277923584
epoch: 12  train_loss: 15.23391056060791
epoch: 13  train_loss: 15.205641746520996
epoch: 14  train_loss: 14.716923713684082
epoch: 15  train_loss: 14.3939208984375
epoch: 16  train_loss: 15.067134857177734
epoch: 17  train_loss: 13.788334846496582
epoch: 18  train_loss: 13.365486145019531
epoch: 19  train_loss: 13.073370933532715
epoch: 20  train_loss: 13.448643684387207
epoch: 21  train_loss: 13.225969314575195
epoch: 22  train_loss: 12.926955223083496
epoch: 23  train_loss: 13.177831649780273
epoch: 24  tra

<All keys matched successfully>

In [19]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = output["test_x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_nm_50-2.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 371.65it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      9.602252          0.000048      0.002301
std       6.035786          0.001523      0.000689
min       0.118130          0.000000      0.000996
25%       4.764137          0.000000      0.002000
50%       9.106999          0.000000      0.002005
75%      12.689323          0.000000      0.002566
max      35.417544          0.048146      0.006606
Number of infeasible solution: 1
